<a href="https://colab.research.google.com/github/kozyreviva/Data_Science/blob/main/timeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.
​
Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.
​
Вам нужно:
​
1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.
​

## Загрузка данных

Подключим библиотеки.

In [1]:
#библиотеки
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.graph_objects as go

Скачаем датасет и сохраним в переменную.

In [2]:
#скачивание датасета
try:
  df = pd.read_csv("/datasets/taxi.csv", index_col = [0],parse_dates = [0])
except:
  df = pd.read_csv("taxi.csv", index_col = [0], parse_dates = [0])

Выведем датасет и посмотрем краткую информацию.

In [3]:
#вывод таблицы
df.head()

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32


In [4]:
#информация о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB


In [5]:
#размер таблицы
df.shape

(26496, 1)

## Анализ данных

Отсортируем таблицу и сгруппируем данные по часу.

In [6]:
#сортировка датафрейма
df.sort_index(inplace=True)
#ресемплирование
df = df.resample('1H').sum()

Проведем небольшой анализ данных.

Т.к данных много для наглядности ресемплируем данные по 6 часов и посмотрим на график

In [7]:
fig = px.line(df.resample("6H").sum(), title = "Количество вызовов")
fig.show()

Посмотрим на тренд и сезонность. Сохраним их в отдельные столбцы.

In [8]:
#создание объекта структуры с результатами применения функции
decomposed = seasonal_decompose(df)
#создание столбцов
df["trand"] = decomposed.trend
df["season"] = decomposed.seasonal


Вывод тренда.

In [9]:
fig = px.line(df["trand"], title = "Тренд")
fig.show()

Вывод сезонности. Возьмем суточный интервал.

In [10]:
fig = px.line(df["2018-03-01":"2018-03-02"]["season"], title = "Сезонность")
fig.show()

Построим график дневной сезонности.

In [11]:
fig = px.line(df["2018-03-01":"2018-03-07"]["season"], title = "Сезонность")
fig.show()

**Выводы:**  

Данные в таблице содержаться всего за пол года, поэтому с точностью нельзя сказать что у нас имеется восходящий тренда за счет роста компании. Возможно количество закаова растёт в летний период.
Суточная сезонность показывает увеличение заказов в полночь и падение к 6 часам. Дневная сезонность не показыевает различий среди недели.

## Обучение моделей

Создадим функцию которая добавляет в датафрейм столбцы с годом, месяцем, днём и днём недели. Также добавим столбцы со сдвигами и скользящим средним.

In [12]:
#функция добавления столбцов
def make_features(data, lag, rolling_mean_size):
    data["hour"] = data.index.day
    data["dayofweek"] = data.index.dayofweek

    for i in range(1, lag+1):
        data["lag_{}",format(i)] = data["num_orders"].shift(i)
    
    data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()


Применение этой функции необходимо для обучения модели. Она предоставляет больше данных.  

Применим функцию.

In [13]:
#применение функции
make_features(df, 2, 24)

Удалим строки с пропусками образовавшиеся в ходе сдвига данных.

In [14]:
#удаление строк
df = df.dropna(axis = 0)

Разделим таблицу на обучающую и тестовую выборки при этом данные не смешиваем.  
Разделим на признаки.

In [15]:
#разделение на выборки
train, test = train_test_split(df, shuffle = False, test_size = 0.1)

#разделение на признаки
features_train = train.drop("num_orders",axis = 1)
target_train = train["num_orders"]

features_test = test.drop("num_orders",axis = 1)
target_test = test["num_orders"]

Создадим валидатор для временных рядов.

In [16]:
#создание временного валидатора
ts = TimeSeriesSplit(n_splits = 8, test_size = 3)

Протестируем модели кросс-валидацией.

In [17]:
%%time
#модель дерева решений
model_tree = DecisionTreeRegressor(random_state = 12345)

params = [{"max_depth": np.arange(1,21,2)}]
gs_tree = GridSearchCV(estimator = model_tree,
                      param_grid=params,
                      scoring="neg_root_mean_squared_error",
                      cv=ts)
gs_tree.fit(features_train.values, target_train.values)
print(gs_tree.best_params_)
print(np.abs(gs_tree.score(features_train.values, target_train.values)))

{'max_depth': 3}
26.13391868149803
CPU times: user 1.41 s, sys: 10.4 ms, total: 1.42 s
Wall time: 1.41 s


In [ ]:
%%time
#модель случайного леса
model_rf = RandomForestRegressor(random_state = 12345)

params = [{"n_estimators": np.arange(10,51,10),
      "max_depth": np.arange(1,16,2)}]
gs_rf = GridSearchCV(estimator = model_rf,
                      param_grid=params,
                      scoring="neg_root_mean_squared_error",
                      cv=ts)
gs_rf.fit(features_train.values, target_train.values)
print(gs_rf.best_params_)
print(np.abs(gs_rf.score(features_train.values, target_train.values)))

In [ ]:
%%time
#линейная регрессия
linear_model = LinearRegression()

scores = cross_val_score(linear_model, features_train.values, target_train.values, scoring="neg_root_mean_squared_error", cv = ts)

print(np.abs(scores.mean()))

In [ ]:
%%time
#модель LightGBM
model_LGBM = LGBMRegressor()

params = [{"n_estimators": np.arange(10,31,10),
      "max_depth": np.arange(1,11,2)}]
gs_LGBM = GridSearchCV(estimator = model_LGBM,
                      param_grid=params,
                      scoring="neg_root_mean_squared_error",
                      cv=ts)
gs_LGBM.fit(features_train.values, target_train.values)
print("LightGBM")
print(gs_LGBM.best_params_)
print(np.abs(gs_LGBM.score(features_train.values, target_train.values)))

## Проверка моделей

In [ ]:
xrange = target_test[:"2018-08-16"].index

Проверим модели на тестовой выборке.

In [ ]:
#модель дерева решений
print("Дерево решений")
print("RMSE:",np.abs(gs_tree.score(features_test.values, target_test.values)))

In [ ]:
predictions = gs_tree.predict(features_test)
fig = go.Figure()
fig.add_trace(go.Scatter(y = target_test, x = xrange,  name = "Реальные",mode='lines',))
fig.add_trace(go.Scatter(y = predictions, x = xrange, name = "Предсказанные",mode='lines',))
fig.update_layout(
    title="Предсказания дерева решений",
    xaxis_title="Дата и время",
    yaxis_title="Количество заказов",
    legend_title="Данные")
fig.show()

In [ ]:
#модель рандомного леса
print("Рандомный лес")
print("RMSE:",np.abs(gs_rf.score(features_test.values, target_test.values)))

In [ ]:
predictions = gs_rf.predict(features_test)
fig = go.Figure()
fig.add_trace(go.Scatter(y = target_test, x = xrange, name = "Реальные",mode='lines',))
fig.add_trace(go.Scatter(y = predictions, x = xrange, name = "Предсказанные",mode='lines',))
fig.update_layout(
    title="Предсказания рандомного леса",
    xaxis_title="Дата и время",
    yaxis_title="Количество заказов",
    legend_title="Данные")
fig.show()

In [ ]:
#модель линейной регрессии
linear_model.fit(features_train.values,target_train.values)
predictions = linear_model.predict(features_test.values)
print("Линейная регрессия")
print("RMSE:",np.sqrt(mean_squared_error(target_test.values, predictions)))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y =target_test, x = xrange, name = "Реальные",mode='lines',))
fig.add_trace(go.Scatter(y =predictions, x = xrange, name = "Предсказанные",mode='lines',))
fig.update_layout(
    title="Предсказания линейной регрессии",
    xaxis_title="Дата и время",
    yaxis_title="Количество заказов",
    legend_title="Данные")
fig.show()

In [ ]:
#модель LightGBM
print("LightGBM")
print("RMSE:",np.abs(gs_LGBM.score(features_test.values, target_test.values)))

In [ ]:
predictions = gs_LGBM.predict(features_test)
fig = go.Figure()
fig.add_trace(go.Scatter(y =target_test, x = xrange, name = "Реальные",mode='lines',))
fig.add_trace(go.Scatter(y =predictions, x = xrange, name = "Предсказанные",mode='lines',))
fig.update_layout(
    title="Предсказания LightGBM",
    xaxis_title="Дата и время",
    yaxis_title="Количество заказов",
    legend_title="Данные")
fig.show()

**Вывод:**  
Модель линейной регрессии дала лучшие результаты по сравнению с другими моделями. НО, если рассматривать данные на графиках, видно что модель LightGBM показывает наибольшую идентичность с реальными данными, а предсказания линейной регрессии расположились более сглаженно. Поэтому, модель lightGBM возможно имеет большую предсказательную ценность.